In [3]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from signal_alignment import phase_align, chisqr_align
from dateutil.parser import *

In [4]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10000)

In [120]:
referenceFile = pd.read_csv('./UVVis/BareSolarLamp2.csv')
referenceFile['intensity'] = referenceFile['intensity'].rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)

spectrumBaseline = referenceFile[referenceFile.wavelength > 200]
spectrumBaseline = spectrumBaseline[referenceFile.wavelength < 238]
spectrumBaseline = spectrumBaseline['intensity'].mean()
referenceFile['intensity'] = referenceFile['intensity'] - spectrumBaseline


referenceFileMaxValue = referenceFile[referenceFile.wavelength > 823.15]
referenceFileMaxValue = referenceFileMaxValue[referenceFile.wavelength < 823.359]
referenceFileMaxValue = referenceFileMaxValue['intensity'].mean()

referenceFile['intensityNorm'] = referenceFile['intensity'] / referenceFileMaxValue

referenceFile = referenceFile.set_index('wavelength')
referenceFile.head()

,Unnamed: 0,intensity,acqTime,time,n,intensityNorm
wavelength,,,,,,
178.997010,0,-3136.829539,11,1.610560e+09,1,-0.402779
179.215281,1,-3136.829539,11,1.610560e+09,1,-0.402779
179.433542,2,-3136.829539,11,1.610560e+09,1,-0.402779
179.651791,3,-3136.829539,11,1.610560e+09,1,-0.402779
179.870030,4,-3136.829539,11,1.610560e+09,1,-0.402779


In [121]:
files = [
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure2.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure3.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure4.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure5.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure6.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure7.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure8.csv'
]

dataframes = []

for fileName in files:
    file = pd.read_csv(fileName)
    file['time'] = file['time'].round(0)
    file['time'] = pd.to_datetime(file['time'], unit='s')
    file['time'] = file['time'] + pd.Timedelta("1 hour")
    file = file.set_index([pd.DatetimeIndex(file['time'])], drop=True)
    file = file.drop(columns='time')
    dataframes.append(file)
    

file = pd.concat(dataframes)

file = file.drop(columns=['Unnamed: 0', 'n'])
file.shape

(4315584, 3)

In [122]:
potentiostatData = pd.read_csv('./UVVis/UVVisCell_Sample04_50nmSi_SolarGen/Potentiostate/MergedFile.csv')
# potentiostatData['actualTime'] = potentiostatData['actualTime'].map(lambda x: parse(x).timestamp())
potentiostatData['time'] = pd.to_datetime(potentiostatData['actualTime'])
potentiostatData = potentiostatData.set_index('time')
potentiostatData['elapsedTime'] = potentiostatData.index - potentiostatData.index.values[0]
potentiostatData = potentiostatData.drop(columns=['Unnamed: 0', 'actualTime', 'T'])
potentiostatData.shape

(73238, 3)

In [213]:
# mergedDataframe = pd.merge_asof(file, potentiostatData, left_index = True, right_index = True, tolerance=pd.Timedelta("1 minutes"))
# mergedDataframe = mergedDataframe.dropna(how='any')
# mergedDataframe

mergedDataframe = pd.merge(left=potentiostatData[0:50000:100], right=file, on='time', how='left')
mergedDataframe = mergedDataframe.dropna(how='any')
# mergedDataframe = mergedDataframe[mergedDataframe['Vf'].notna()]

# mergedDataframe = pd.merge_asof(file, potentiostatData, left_index=True, right_index=True, tolerance=pd.Timedelta("1 seconds"))
# mergedDataframe = mergedDataframe.dropna(how='any')
mergedDataframe.shape

(51072, 6)

In [214]:
mergedDataframe

,Vf,Im,elapsedTime,wavelength,intensity,acqTime
time,,,,,,
2021-01-18 15:51:04,0.762531,-0.000003,0 days 01:21:40,178.997010,2828.078243,5474.0
2021-01-18 15:51:04,0.762531,-0.000003,0 days 01:21:40,179.215281,2828.078243,5474.0
2021-01-18 15:51:04,0.762531,-0.000003,0 days 01:21:40,179.433542,2828.078243,5474.0
2021-01-18 15:51:04,0.762531,-0.000003,0 days 01:21:40,179.651791,2749.520514,5474.0
2021-01-18 15:51:04,0.762531,-0.000003,0 days 01:21:40,179.870030,2860.215496,5474.0
...,...,...,...,...,...,...
2021-01-18 19:54:13,0.606328,-0.000002,0 days 05:24:49,884.693799,5784.705498,16405.0
2021-01-18 19:54:13,0.606328,-0.000002,0 days 05:24:49,884.858069,5640.087860,16405.0
2021-01-18 19:54:13,0.606328,-0.000002,0 days 05:24:49,885.022319,5547.246908,16405.0


In [215]:
# mergedDataframe['changed'] = mergedDataframe["wavelength"].shift(-1).isna() != mergedDataframe["wavelength"].isna()
# mergedDataframe
# edges = mergedDataframe[mergedDataframe['changed'] == True].index
# edges

# dataframecuts = pd.cut(mergedDataframe.index, bins=edges, right=True, include_lowest=False)
# dataframecuts.categories

# allCutDataframes = []
# for categorie in dataframecuts.categories[0::2]:
#     allCutDataframes.append(mergedDataframe[categorie.left:categorie.right])
    
# mergedDataframeCut = pd.concat(allCutDataframes)
# mergedDataframe = mergedDataframeCut.drop_duplicates()

In [216]:
X = referenceFile.index.values
Y = [date for date in mergedDataframe.groupby('time').groups]

XX, YY = np.meshgrid(X, Y)

In [217]:
Z_absSmoothed = []
Z_intensityNorm = []
Z_Im = []
Z_V = []

for yGrid in YY:
    try:
        spectrum = mergedDataframe.xs(str(yGrid[0]))
        
        if spectrum.shape[0] == 3648:
        
            spectrum = spectrum.set_index('wavelength')

            spectrum['intensity'] = spectrum['intensity'].rolling(50).mean()

            spectrumBaseline = spectrum[(spectrum.index > 200) & (spectrum.index < 300)]
            spectrumBaseline = spectrumBaseline['intensity'].mean()
            spectrum['intensity'] = spectrum['intensity'] - spectrumBaseline


            spectrumMaxValue = spectrum[(spectrum.index > 823.15) & (spectrum.index < 823.359)]
            spectrumMaxValue = spectrumMaxValue['intensity'].mean()
            spectrum['intensityNorm'] = spectrum['intensity'] / spectrumMaxValue


            spectrum['abs'] = -np.log(spectrum['intensity'] / referenceFile['intensityNorm'])
            spectrum['absSmoothed'] = spectrum['abs'].rolling(50).mean()

            Z_absSmoothed.append(spectrum['absSmoothed'].fillna(0).values)
            Z_intensityNorm.append(spectrum['intensityNorm'].fillna(0).values)
            Z_Im.append(spectrum['Im'].fillna(0).values)
            Z_V.append(spectrum['Vf'].fillna(0).values)
        
        else:
            Z_absSmoothed.append(np.zeros(3648))
            Z_intensityNorm.append(np.zeros(3648))
            Z_Im.append(np.zeros(3648))
            Z_V.append(np.zeros(3648))
            
    
    except KeyError as err:
        continue
        
Z_absSmoothed = np.array(Z_absSmoothed)
Z_intensityNorm = np.array(Z_intensityNorm)
Z_Im = np.array(Z_Im)
Z_V = np.array(Z_V)

In [218]:
figure2 = plt.figure()
figure2ax1 = figure2.add_subplot(1,3,1)
# figure2ax1.set_ylim(0, 1.5)
figure2ax2 = figure2.add_subplot(1,3,2)
# figure2ax2.set_ylim(-0.5, 6)
figure2ax3 = figure2.add_subplot(1,3,3)
# figure2ax2.set_ylim(-0.5, 6)

figure2ax1mappable = figure2ax1.plot(X, Z_absSmoothed.T)
figure2ax2mappable = figure2ax2.plot(X, Z_intensityNorm.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [219]:
# mergedDataframe['elapsedTimeSeconds'] = mergedDataframe['elapsedTime'].dt.seconds/3600


# Y = np.array([date for date in mergedDataframe.groupby('elapsedTimeSeconds').groups])

# XX, YY = np.meshgrid(X, Y)

In [220]:
print(XX.shape)
print(YY.shape)
print(Z_absSmoothed.shape)

(14, 3648)
(14, 3648)
(14, 3648)


In [221]:
figure3 = plt.figure()

figure3ax1 = figure3.add_subplot(1,4,1)
# ax5.set_xlim(265, 807)

figure3ax2 = figure3.add_subplot(1,4,2)
# ax6.set_xlim(265, 807)

figure3ax3 = figure3.add_subplot(1,4,3)
# ax6.set_xlim(265, 807)

figure3ax4 = figure3.add_subplot(1,4,4)
# ax6.set_xlim(265, 807)

cont1 = figure3ax1.pcolormesh(XX, YY, Z_absSmoothed, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=-12, vmax=0)

cont2 = figure3ax2.imshow(Z_intensityNorm, aspect='auto', vmin=0, vmax=2.7)

cont3 = figure3ax3.imshow(Z_V, aspect='auto',  vmin=0.01, vmax=0.8)

cont4 = figure3ax4.imshow(Z_Im, cmap=cm.plasma, aspect='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import mpl_scatter_density

figure4 = plt.figure(figsize=(15, 15))

figure4ax1 = figure4.add_subplot(4,2,1)
figure4ax2 = figure4.add_subplot(4,2,3, sharex=figure4ax1)
figure4ax3 = figure4.add_subplot(4,2,5, sharex=figure4ax1)
figure4ax4 = figure4.add_subplot(4,2,7, sharex=figure4ax1)
figure4ax5 = figure4.add_subplot(4,2,2)
figure4ax6 = figure4.add_subplot(4,2,4, sharex=figure4ax5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
mappable1 = figure4ax1.scatter(mergedDataframe.index, mergedDataframe['wavelength'], c=Z_absSmoothed, s=0.001)
cbar1 = figure4.colorbar(ax=figure4ax1, orientation='horizontal', mappable=mappable1)

ValueError: 'c' argument has 18240000 elements, which is inconsistent with 'x' and 'y' with size 1339802.

In [13]:
mappable2 = figure4ax2.scatter(mergedDataframe.index, mergedDataframe['wavelength'], c=Z_intensityNorm, s=0.001, vmin=0, vmax=2.5)
cbar2 = figure4.colorbar(ax=figure4ax2, orientation='horizontal', mappable=mappable2, ticks=[0, 1, 2.5])

In [14]:
figure4ax3.plot(mergedDataframe.index, mergedDataframe['Vf'])
figure4ax4.plot(mergedDataframe.index, mergedDataframe['Im'])

In [73]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools as it

from matplotlib.gridspec import GridSpec


from matplotlib import style


# style.use('fivethirtyeight')
# style.use('fast')
# style.use('seaborn-deep')
style.use('default')

figure = plt.figure(figsize=(15, 15))



gridspec = GridSpec(4, 3, width_ratios=[1, 0.05, 1], height_ratios=[1, 1, 1, 1], wspace=0.2)
gridspec2 = GridSpec(4, 3, width_ratios=[1, 0.05, 1], height_ratios=[1, 1, 1, 1], wspace=0, hspace=0)

subplotspec1 = gridspec2.new_subplotspec((0, 0), rowspan=1, colspan=1)
subplotspec2 = gridspec2.new_subplotspec((1, 0), rowspan=1, colspan=1)
subplotspec3 = gridspec2.new_subplotspec((2, 0), rowspan=1, colspan=1)
subplotspec4 = gridspec2.new_subplotspec((3, 0), rowspan=1, colspan=1)
subplotspec5 = gridspec.new_subplotspec((0, 1), rowspan=1, colspan=1)
subplotspec6 = gridspec.new_subplotspec((1, 1), rowspan=1, colspan=1)
subplotspec7 = gridspec.new_subplotspec((0, 2), rowspan=1, colspan=1)
subplotspec8 = gridspec.new_subplotspec((1, 2), rowspan=1, colspan=1)


ax1 = figure.add_subplot(subplotspec1)
ax1.tick_params(labelbottom=False)
ax1.set_ylabel('Wavelength')
ax1.set_ylim(200, 850)


ax5 = figure.add_subplot(subplotspec5)
ax1mappable = ax1.scatter(mergedDataframe.index, mergedDataframe['wavelength'], c=Z_absSmoothed, s=0.001)
cbar1 = figure.colorbar(ax=ax1, cax=ax5, orientation='vertical', mappable=ax1mappable)
cbar1.ax.set_ylabel('Normalized Counts')




ax2 = figure.add_subplot(subplotspec2, sharex=ax1)
ax2.tick_params(labelbottom=False)
ax2.set_ylabel('Wavelength')
ax2.set_ylim(200, 850)


ax6 = figure.add_subplot(subplotspec6)
ax2mappable = ax2.scatter(mergedDataframe.index, mergedDataframe['wavelength'], c=Z_intensityNorm, s=0.001, vmin=0, vmax=2.5)
cbar2 = figure.colorbar(ax=ax2, cax=ax6, orientation='vertical', mappable=ax2mappable, ticks=[0, 1, 2.5])
cbar2.ax.set_ylabel('Absorption')

ax3 = figure.add_subplot(subplotspec3, sharex=ax1)
ax3.tick_params(labelbottom=False)
ax3.set_ylabel('Current')
ax3mappable = ax3.plot(mergedDataframe.index, mergedDataframe['Vf'])


ax4 = figure.add_subplot(subplotspec4, sharex=ax1)
ax4.set_xlabel('Time')
ax4.set_ylabel('Potential')
ax4mappable = ax4.plot(mergedDataframe.index, mergedDataframe['Im'])


ax7 = figure.add_subplot(subplotspec7, sharey=ax1)
ax7.set_xlabel('Absorption')
ax7.set_ylabel('Wavelength')
ax7.mappable = ax7.plot(Z_absSmoothed[::36].T, X)
ax7.set_xlim(-12, -2)
ax7.yaxis.set_ticks_position('right')
ax7.yaxis.set_label_position('right')

ax8 = figure.add_subplot(subplotspec8, sharey=ax2)
ax8.set_xlabel('Normalized Counts')
ax8.set_ylabel('Wavelength')
ax8.mappable = ax8.plot(Z_intensityNorm[40::100].T, X)
ax8.set_xlim(0, 2.7)
ax8.yaxis.set_ticks_position('right')
ax8.yaxis.set_label_position('right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
figureTest = plt.figure(figsize=(7, 7))
axwhatever = figureTest.add_subplot(1,1,1)
axwhatever.plot(Z_intensityNorm[40::100].T, X)
axwhatever.set_ylim(282, 810)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(282.0, 810.0)